This notebook has the cleaned versions of dfs you have created with all the boolean columns as well.
- Class Id was also added to the dfs for easier play with merging.
- Refer to 2_pickle notebook for how the pickle files were created.
- 2_pickle also has questions you may want to answer for further data exploration/analysis

**This notebook focuses on getting Spotify artist data based on Kaggle dataset 1 as well as using spotipy to
pull data on any unmatched artists. Spoiler alert - there was a larger Kaggle dataset that was used instead. 
Skip this notebook and go to notebook 4_...v2**

In [105]:
# Basics 
import pandas as pd
import numpy as np

import math

# Text Cleaning
import re

#Spotify
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time 

# Importing dfs with heavy processing
import pickle

In [106]:
# import peloton data
# Thank you to okaykristinakay on Reddit for the data!
# Source: https://www.reddit.com/r/pelotoncycle/comments/m18xnr/peloton_class_list_march_update/
df = pd.read_excel('../AGF_Peloton Classes March.xlsx')
df.head()

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,None,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...


In [107]:
# Opening pickle files
# Use classId where you can as key for merging
base_clean_df = pd.read_pickle("./base_clean_df.pkl")
artist_df = pd.read_pickle("./artist_df.pkl")
artist_class_count_df = pd.read_pickle("./artist_class_count_df.pkl")
songs_df = pd.read_pickle("./songs_df.pkl")
song_class_count_df = pd.read_pickle("./song_class_count_df.pkl")
class_type_reorg_df = pd.read_pickle("./class_type_reorg_df.pkl")
new_class_types_with_artists = pd.read_pickle("./new_class_types_with_artists.pkl")

In [108]:
base_clean_df
# Difference from df:
# Dropped null 'instructorBio' rows - only 9
# Dropped rows with null classSongs/classArtists
# Dropped row value with classSongs as float (1 row)
# Populate rows with classType Cycling and classEquipment None to classEquipment = Bike

,classId,className,classDescription,classDifficulty,classDuration,classType,classLength,classLocation,classOriginalAirdate,classRating,classRatingCount,instructorName,instructorBio,classEquipment,classSongs,classArtists,classUrl
0,7f66378211c9476b9b5619bf989f91d0,20 min Peace Meditation,A guided meditation that focuses on cultivatin...,4.3333,20,Meditation,23,psny-studio-2,2021-09-03 13:25:00,0.9847,131,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...",Yoga Block,Meditation 22,RIOPY,https://members.onepeloton.com/classes/bootcam...
1,54ac61803b364b2fa8378acd9f593cdb,15 min Bodyweight Strength,"No equipment, no problem. Join us for a high-e...",5.7755,15,Strength,19,psny-studio-3,2021-09-03 13:19:00,0.9933,297,Olivia Amato,"Born and raised in New York, Olivia grew up pl...",Workout Mat,"California Gurls,Let's Get Loud,Let It Rock (f...","Katy Perry,Snoop Dogg,Jennifer Lopez,Kevin Rud...",https://members.onepeloton.com/classes/bootcam...
2,c75fd4831573483c9d45739aae11d083,20 min Focus Flow: Lower Body,This yoga flow class focuses on poses that eng...,4.3664,20,Yoga,23,psny-studio-2,2021-09-03 12:51:00,1.0000,159,Aditi Shah,"To Aditi, yoga goes beyond movement and can br...","Yoga Blanket,Yoga Block,Yoga Mat","Interlude No 1,Oceansize,She Just Likes To Fig...","James Vincent McMorrow,Oh Wonder,Four Tet,Grim...",https://members.onepeloton.com/classes/bootcam...
3,470086936f7a4723ab5a53cb80b571ff,45 min Pop Bootcamp,Split your workout 50/50 between cardio on the...,7.8312,45,Tread Bootcamp,50,psny-studio-4,2021-09-03 11:56:00,0.9737,152,Olivia Amato,"Born and raised in New York, Olivia grew up pl...","Workout Mat,Medium Weights",34+35 (Remix) (feat. Doja Cat & Megan Thee Sta...,"Ariana Grande,Doja Cat,Megan Thee Stallion,Jus...",https://members.onepeloton.com/classes/bootcam...
4,9680a817bf2149d2b91990c87166a400,20 min Pop Ride,We dare you not to dance as you ride to all th...,7.4000,20,Cycling,24,uk,2021-09-03 07:52:00,1.0000,82,Sam Yo,Sam is a pro at many things but shines when it...,Bike,"Señorita,Marry You,Irreplaceable,What Do You M...","Justin Timberlake,Bruno Mars,Beyoncé,Justin Bi...",https://members.onepeloton.com/classes/bootcam...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16743,a144fcf1595740caa610f6484deb3cdb,20 min Funk Fun Run,This 20 minute Fun Run is a musically driven c...,6.4567,20,Running,23,nyc-gw-tread,14/09/2018 10:22,0.9933,150,Becs Gentry,Becs joins Peloton from London as an accomplis...,None,"Man Funk (feat. Leron Thomas),If It Ain't Funk...","Guts,Leron Thomas,Soopasoul,FDEL,Basement Frea...",https://members.onepeloton.com/classes/bootcam...
16744,bd89a56c884b48e59ce9c43ab4f4d86f,45 min 90s Pop Ride,The 90s were a booming and inventive time in m...,8.1481,45,Cycling,49,nyc,13/09/2018 23:19,0.9925,1877,Denis Morton,"Raised in Florida, schooled in Tennessee, stee...",Light Weights,"Too Close - Radio Edit,Motownphilly - Original...","Next,Boyz II Men,Blues Traveler,Eagle-Eye Cher...",https://members.onepeloton.com/classes/bootcam...
16745,e5a99c83d7fc4f0686c68b02e5932204,10 min Arms Toning,Join Jess as she takes you through this 10-min...,7.4028,10,Strength,11,nyc,13/09/2018 21:02,0.9978,4481,Jess King,Jess is a charismatic instructor with a boundl...,Light Weights,"Hustle,Day and Night,Just Got Paid,Waiting for...","Daphne Willis,Gizzle,Lo Air,Sigala,Ella Eyre,M...",https://members.onepeloton.com/classes/bootcam...
16746,c57e123004d04df1bfae94c5e4399bb3,45 min 90s Ride,The 90s were a booming and inventive time in m...,8.3982,45,Cycling,49,nyc,13/09/2018 11:53,0.9922,2307,Emma Lovewell,Emma Lovewell is a Martha’s Vineyard native an...,Light Weights,"Show Me Love - Radio Version,Don't Speak,Gonna...","Robyn,No Doubt,C & C Music Factory,Technotroni...",https://members.onepeloton.com/classes/bootcam...


Note:
classId = 8840f82cdc624aa28a5d1babc51a3916 is the one you need to watch out for (has float classSong)

In [109]:
# Check lengths of dfs
print(f'artist_df: {len(artist_df)}')
print(f'artist_class_count_df: {len(artist_class_count_df)}')
print(f'songs_df: {len(songs_df)}')
print(f'song_class_count_df: {len(song_class_count_df)}')
print(f'class_type_reorg_df: {len(class_type_reorg_df)}')
print(f'new_class_types_with_artists: {len(new_class_types_with_artists)}')
# No inconsistencies where there shouldn't be

artist_df: 16748
artist_class_count_df: 10941
songs_df: 16748
song_class_count_df: 29904
class_type_reorg_df: 16748
new_class_types_with_artists: 16748


#### Before cleaning Artist List try to match to Kaggle Spotify df for artist IDs

In [110]:
artist_class_count_df = artist_class_count_df.sort_values(by='Artist').reset_index(drop=True)

In [111]:
artist_class_count_df.rename(columns={'Artist':'artist'}, inplace=True)

In [112]:
artist_class_count_df

,artist,Class Count
0,ACE HOOD,1
1,AKON,1
2,Cambridge,1
3,Cr3on,6
4,DJ Deeon,13
...,...,...
10936,yutaka hirasaka,1
10937,Álvaro Faria,4
10938,Ásgeir,12
10939,Ådå,1


In [113]:
# Import Kaggle Spotify artist IDs for matching
# Source: https://www.kaggle.com/ehcall/spotify-artists
spot_artist_df = pd.read_csv('artist-uris.csv', names=['artist', 'spotifyId'])
spot_artist_df.head()

,artist,spotifyId
0,1:43,spotify:artist:39EHxSQAIaWusRqSI9xoyF
1,2:00 AM,spotify:artist:4tN3rZ7cChj4Wns2Wt2Nj6
2,2:15,spotify:artist:4HsOm6VNKZtGh8W8GhdNu4
3,2:54,spotify:artist:3LsQKoRgMc8VEkQn66jfAQ
4,4:20,spotify:artist:5KCG0FDMDPzQpxcohGUnyH


In [114]:
artist_class_count_spotid = artist_class_count_df.merge(spot_artist_df, on='artist', how='left', indicator=True)
# 71% of artists matched with spotifyId with inner join

In [115]:
artist_class_count_spotid['_merge'].value_counts()
# Total is up to 11282l; duplicate rows added
# More than one Spotify id for an artist

both          7803
left_only     3479
right_only       0
Name: _merge, dtype: int64

In [116]:
artist_class_count_spotid[artist_class_count_spotid['artist'].duplicated(keep='first')].sort_values(by='artist')['artist']

136      A Taste Of Honey
170               ARMNHMR
187                    AZ
198          Aaron Watson
246        Adventure Club
               ...       
11174                Zion
11173                Zion
11172                Zion
11175                Zion
11191                 Zoé
Name: artist, Length: 341, dtype: object

In [117]:
# Drop duplicate rows, take first spotify id matched with for 341 rows
artist_class_count_spotid.drop_duplicates(subset='artist', keep='first', inplace=True)

In [118]:
# Reset index post drops
artist_class_count_spotid = artist_class_count_spotid.reset_index(drop=True)

In [119]:
artist_class_count_spotid

,artist,Class Count,spotifyId,_merge
0,ACE HOOD,1,NaN,left_only
1,AKON,1,NaN,left_only
2,Cambridge,1,NaN,left_only
3,Cr3on,6,NaN,left_only
4,DJ Deeon,13,NaN,left_only
...,...,...,...,...
10936,yutaka hirasaka,1,NaN,left_only
10937,Álvaro Faria,4,NaN,left_only
10938,Ásgeir,12,spotify:artist:7xUZ4069zcyBM4Bn10NQ1c,both
10939,Ådå,1,NaN,left_only


In [120]:
artist_class_count_spotid['spotifyId'].isnull().sum()
# ~3500 artists left to match

3479

In [121]:
len(artist_class_count_spotid)

10941

In [122]:
leftover_artists = list(artist_class_count_spotid[artist_class_count_spotid['spotifyId'].isnull()]['artist'])

Clean leftover artists for Spotify API work

In [123]:
# Remove leading and trailing spaces from values
leftover_artists = [x.strip() for x in leftover_artists]

In [124]:
# Remove punctuation
leftover_artists = [re.sub(r'[^\w\s]', '', (x)) for x in leftover_artists]

In [125]:
leftover_artists = sorted(leftover_artists)

In [126]:
# Spaces after punctuation removed, re-strip
leftover_artists = [x.strip() for x in leftover_artists]

In [127]:
# First value is a blank space
leftover_artists = leftover_artists[1:]

In [128]:
# Noticed 10,000 maniacs artists was split; spot clean
leftover_artists.remove('10')
leftover_artists[0] = '10000 maniacs'

In [129]:
len(leftover_artists)

3477

### Spotify API Work

In [130]:
#Spotify API set up
# Source: https://betterprogramming.pub/how-to-extract-any-artists-data-using-spotify-s-api-python-and-spotipy-4c079401bc37
client_id = '4aac5b215c8a4fc591b506b71af7ebf5'
client_secret = '31d999b609424f569382b047ba828b00'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [131]:
leftover_artists

['10000 maniacs',
 'Kyle',
 'L',
 '000 Maniacs',
 '1 Mile North',
 '1 World',
 '10cc',
 '112',
 '1991',
 '2014 Remastered Version',
 '2016 Remaster',
 '20syl',
 '220 KID',
 '2220s',
 '24 Karatz',
 '24Heavy',
 '24KGoldn',
 '24hrs',
 '24kGoldn',
 '2CELLOS',
 '2gether',
 '3 Of A Kind',
 '30 Seconds To Mars',
 '311',
 '369',
 '3LAU',
 '3LW',
 '40 Glocc',
 '40 Thevz',
 '433',
 '44',
 '4AM',
 '4B',
 '4FRNT',
 '6AM',
 '6ix9ine',
 '7 Version',
 '702',
 '70s Greatest Hits',
 '70s Music',
 '80s Hits',
 '80s Pop',
 '88GLAM',
 '88rising',
 '90s Maniacs',
 '90s Pop Band',
 '90s Throwback',
 '95 North',
 '999',
 '9Tendo',
 'A Manetta',
 'A Rocket To the Moon',
 'A Touch Of Class',
 'A7S',
 'AB Soto',
 'ACE HOOD',
 'ADAM',
 'AG',
 'AGTrio',
 'AHS Project',
 'AIR',
 'AJ Shively',
 'AKON',
 'ANT LaROCK',
 'AP3',
 'APM Music',
 'APRE',
 'APeace',
 'ARTY',
 'ASCO',
 'ASM',
 'ATFC',
 'AUDREY NUNA',
 'AUDÉ',
 'AVAN LAVA',
 'AWE',
 'AWE Remix',
 'AY AY',
 'AYOR',
 'AZTX',
 'Aaron Lindt',
 'Abel Aguilera',
 

In [132]:
# Set up search term 
searchtest = leftover_artists[0]
searchtest

'10000 maniacs'

In [133]:
# Pull out artistid
result = sp.search(searchtest)
artistidtest = result['tracks']['items'][0]['album']['artists'][0]['uri']
artistidtest

'spotify:artist:0MBIKH9DjtBkv8O3nS6szj'

In [134]:
# leftover_artists_df = pd.DataFrame(leftover_artists, columns=['artist'])

In [135]:
# # Create function to get artist id
# def getartistId(artist):
#     searchterm = artist
#     try:
#         result = sp.search(searchterm)
#         artistid = result['tracks']['items'][0]['album']['artists'][0]['uri']
#     except: # if do not find match, error
#         return "error"
#         pass
#     else: # if do find match, return id
#         return artistid

In [136]:
# leftover_artists_df['spotifyId_man'] = leftover_artists_df['artist'].map(lambda a: getartistId(a))

In [137]:
# leftover_artists_df

In [138]:
# leftover_artists_df['spotifyId_man'].value_counts()
# # spotify:artist:0LyfQWJT6nXafLPZqxe9Of = 'Various Artists'
# # 37 errors
# # 96% of artists identified out of original 10941

In [140]:
# Bring in leftover artists df from pickle file
# spotify:artist:0LyfQWJT6nXafLPZqxe9Of = 'Various Artists'
# 37 errors
leftover_artists_df = pd.read_pickle("./leftover_artists_df.pkl")

In [141]:
leftover_artists_df

,artist,spotifyId_man
0,10000 maniacs,spotify:artist:0MBIKH9DjtBkv8O3nS6szj
1,Kyle,spotify:artist:4qBgvVog0wzW75IQ48mU7v
2,L,spotify:artist:0du5cEVh5yTK9QJze8zA0C
3,000 Maniacs,spotify:artist:0MBIKH9DjtBkv8O3nS6szj
4,1 Mile North,spotify:artist:2e62yyjw0GPB4heVGRwczC
...,...,...
3472,twocolors,spotify:artist:7ACEUD7UsmmXrnj4OLt8f9
3473,upper class,spotify:artist:2NtGuhjeGjxetrptLSQHV0
3474,yutaka hirasaka,spotify:artist:0klU31gyC6EZqKgLvVANCy
3475,Álvaro Faria,spotify:artist:3Vop0LjMsp1743VPBhYJXl


In [142]:
# Drop _merge column from original df, prepare for next merge with spotifyId_man column addition
artist_class_count_spotid = artist_class_count_spotid.drop(columns="_merge")

In [143]:
# repeat left join between original artist_class_count_spotid and new leftover_artists_df based on artist
artist_class_count_spotid_2 = artist_class_count_spotid.merge(leftover_artists_df, on='artist', how='left', indicator=True)

In [144]:
# Duplicates added
artist_class_count_spotid_2['_merge'].value_counts()

left_only     7856
both          3092
right_only       0
Name: _merge, dtype: int64

In [146]:
# View duplicate artists
artist_class_count_spotid_2[artist_class_count_spotid_2['artist'].duplicated(keep='first')].sort_values(by='artist')['artist']

2099                Cr3on
5453                    L
5694           Lil Boosie
6140                 Mali
6849             N Douwma
7321              P Diddy
8644    Sherry St Germain
Name: artist, dtype: object

In [147]:
# Drop duplicate rows, take first spotify id matched with for 7 records
artist_class_count_spotid_2.drop_duplicates(subset='artist', keep='first', inplace=True)

In [148]:
# Reset index post drops
artist_class_count_spotid_2 = artist_class_count_spotid_2.reset_index(drop=True)

In [156]:
math.isnan(artist_class_count_spotid_2['spotifyId'][0])

True

In [168]:
# Replace nans in ['spotifyId'] with values found in manual Id pull (['spotifyId_man'])
artist_class_count_spotid_2['spotifyId'] = np.where(~artist_class_count_spotid_2['spotifyId'].isnull(),
                                                    artist_class_count_spotid_2['spotifyId'],artist_class_count_spotid_2['spotifyId_man'])

In [172]:
# Clean df by dropping 'spotifyId_man', '_merge' helper columns
artist_class_count_spotid_2.drop(columns=['spotifyId_man', '_merge'], inplace=True)

In [182]:
1-((artist_class_count_spotid_2['spotifyId'].isnull().sum())/len(artist_class_count_spotid_2))
# 96% of artists have spotify Id

0.9622520793346129

In [184]:
artist_class_count_spotid_2

,artist,Class Count,spotifyId
0,ACE HOOD,1,NaN
1,AKON,1,NaN
2,Cambridge,1,NaN
3,Cr3on,6,NaN
4,DJ Deeon,13,NaN
...,...,...,...
10936,yutaka hirasaka,1,spotify:artist:0klU31gyC6EZqKgLvVANCy
10937,Álvaro Faria,4,spotify:artist:3Vop0LjMsp1743VPBhYJXl
10938,Ásgeir,12,spotify:artist:7xUZ4069zcyBM4Bn10NQ1c
10939,Ådå,1,spotify:artist:6FBDaR13swtiWwGhX1WQsP


# VOID - Old code for pinging Spotify for artists info

In [97]:
# # Get list of artists from artist_class_count_df
# all_artists_list = list(artist_class_count_df.sort_values(by='Artist')['Artist'])

Clean values

In [98]:
# # Remove leading and trailing spaces from values
# all_artists_list = [x.strip() for x in all_artists_list]

In [106]:
# all_artists_list = sorted(all_artists_list)
# all_artists_list

['" Kyle',
 '" L "',
 '"Cats" 1983 Broadway Cast',
 '"Fiddler On The Roof” Motion Picture Orchestra',
 '"Jesus Christ Superstar" 1996 London Cast',
 '"The Bodyguard the Musical" World Premiere Cast',
 '"The Phantom Of The Opera" Original London Cast',
 '#Mali',
 "'Come From Away' Company",
 "'Hair' Tribe",
 '(Bonus Track)',
 '(Hed) Planet Earth',
 '*NSYNC',
 '+44',
 '....',
 '.JETTA',
 '000 Maniacs',
 '1 Mile North',
 '1 World',
 '10',
 '10 Years',
 '101 Strings Orchestra',
 '10cc',
 '112',
 '12th Planet',
 '1991',
 '2 Brothers On The 4th Floor',
 '2 Chainz',
 '2 Pistols',
 '2 Unlimited',
 '2014 Remastered Version',
 '2016 Remaster',
 '20syl',
 '21 Savage',
 '22-20s',
 '220 KID',
 '24 Karatz',
 '24Heavy',
 '24KGoldn',
 '24hrs',
 '24kGoldn',
 '2CELLOS',
 '2gether',
 '3 Doors Down',
 '3 Of A Kind',
 '30 Seconds To Mars',
 '311',
 '369',
 '38 Special',
 '3LAU',
 '3LW',
 '3OH!3',
 '4 Non Blondes',
 '4 P.M.',
 '4 Strings',
 '40 Glocc',
 '40 Thevz',
 '42nd Street Ensemble',
 '433',
 '4AM',
 

In [107]:
# len(all_artists_list)

10941

Cleaning artist list for Spotify searching

In [100]:
# # Remove punctuation
# all_artists_list = [re.sub(r'[^\w\s]', '', (x)) for x in all_artists_list]

In [ ]:
# all_artists_list = sorted(all_artists_list)

In [75]:
# # Spaces after punctuation removed, re-strip
# all_artists_list = [x.strip() for x in all_artists_list]

In [76]:
# # First value is a blank space
# all_artists_list = all_artists_list[1:]

In [78]:
# # Restablish list with unique values
# all_artists_list = sorted(set(all_artists_list))

In [79]:
# len(all_artists_list)
# # Removed/combined 123 values

10818

In [80]:
# all_artists_list

['000 maniacs',
 '1 mile north',
 '1 world',
 '10',
 '10 years',
 '101 strings orchestra',
 '10cc',
 '112',
 '12th planet',
 '1991',
 '2 brothers on the 4th floor',
 '2 chainz',
 '2 pistols',
 '2 unlimited',
 '2014 remastered version',
 '2016 remaster',
 '20syl',
 '21 savage',
 '220 kid',
 '2220s',
 '24 karatz',
 '24heavy',
 '24hrs',
 '24kgoldn',
 '2cellos',
 '2gether',
 '3 doors down',
 '3 of a kind',
 '30 seconds to mars',
 '311',
 '369',
 '38 special',
 '3lau',
 '3lw',
 '3oh3',
 '4 non blondes',
 '4 pm',
 '4 strings',
 '40 glocc',
 '40 thevz',
 '42nd street ensemble',
 '433',
 '44',
 '4am',
 '4b',
 '4frnt',
 '5 seconds of summer',
 '50 cent',
 '5dime',
 '69 boyz',
 '6am',
 '6ix9ine',
 '6lack',
 '7 sons of soul',
 '7 version',
 '702',
 '70s greatest hits',
 '70s music',
 '7th heaven',
 '808 state',
 '80s hits',
 '80s pop',
 '88glam',
 '88rising',
 '90s maniacs',
 '90s pop band',
 '90s throwback',
 '95 north',
 '98º',
 '99 souls',
 '999',
 '9tendo',
 'a  z',
 'a boogie wit da hoodie',

In [81]:
# # Notices 10,000 maniacs artists was split; spot clean
# all_artists_list.remove('10')
# all_artists_list[0] = '10000 maniacs'

In [82]:
# all_artists_list

['10000 maniacs',
 '1 mile north',
 '1 world',
 '10 years',
 '101 strings orchestra',
 '10cc',
 '112',
 '12th planet',
 '1991',
 '2 brothers on the 4th floor',
 '2 chainz',
 '2 pistols',
 '2 unlimited',
 '2014 remastered version',
 '2016 remaster',
 '20syl',
 '21 savage',
 '220 kid',
 '2220s',
 '24 karatz',
 '24heavy',
 '24hrs',
 '24kgoldn',
 '2cellos',
 '2gether',
 '3 doors down',
 '3 of a kind',
 '30 seconds to mars',
 '311',
 '369',
 '38 special',
 '3lau',
 '3lw',
 '3oh3',
 '4 non blondes',
 '4 pm',
 '4 strings',
 '40 glocc',
 '40 thevz',
 '42nd street ensemble',
 '433',
 '44',
 '4am',
 '4b',
 '4frnt',
 '5 seconds of summer',
 '50 cent',
 '5dime',
 '69 boyz',
 '6am',
 '6ix9ine',
 '6lack',
 '7 sons of soul',
 '7 version',
 '702',
 '70s greatest hits',
 '70s music',
 '7th heaven',
 '808 state',
 '80s hits',
 '80s pop',
 '88glam',
 '88rising',
 '90s maniacs',
 '90s pop band',
 '90s throwback',
 '95 north',
 '98º',
 '99 souls',
 '999',
 '9tendo',
 'a  z',
 'a boogie wit da hoodie',
 'a 

In [83]:
# # Set up search term 
# searchtest = all_artists_list[0]
# searchtest

'10000 maniacs'

In [84]:
# # Pull out trackid
# result = sp.search(searchtest)
# # trackidtest = result['tracks']['items'][0]['id']
# result

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=10000+maniacs&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0MBIKH9DjtBkv8O3nS6szj'},
       'href': 'https://api.spotify.com/v1/artists/0MBIKH9DjtBkv8O3nS6szj',
       'id': '0MBIKH9DjtBkv8O3nS6szj',
       'name': '10,000 Maniacs',
       'type': 'artist',
       'uri': 'spotify:artist:0MBIKH9DjtBkv8O3nS6szj'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',